# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [1]:
import pandas as pd
import numpy as np
import folium


In [2]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv('../../DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)
print(np.shape(X),np.shape(y))

(400000, 31) (400000,)


### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [3]:
# install 
!pip install scikit-optimize

### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [4]:
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy()[:50000,:], y.to_numpy()[:50000])

In [5]:
#linear regression
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(X_train,y_train)
print(linear.score(X_test, y_test))

0.7103211871613051


In [ ]:
#random forest regression
from sklearn.ensemble import RandomForestRegressor
optimizer = BayesSearchCV(RandomForestRegressor(),{'n_estimators':[10, 20, 50, 70, 100, 300],'max_depth':[2, 4, 6, 8, 10, 12, 14]},n_iter=30,random_state=0, n_jobs=-1)
forest = optimizer.fit(X_train, y_train)
forest.score(X_test, y_test)

In [ ]:
forest.best_score_

In [ ]:
forest.best_params_

In [ ]:
#SVR
from sklearn.svm import SVR
svr_optimizer = BayesSearchCV(SVR(),{'C': [1.1, 20.2, 35.3, 55.5, 100.7],'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['rbf'],},n_iter=30,random_state=0)
svr = svr_optimizer.fit(X_train, y_train)
svr.score(X_test, y_test)

In [ ]:
svr.best_score_

In [ ]:
svr.best_params_